In [1]:
!pip install transformers datasets scikit-learn torch beautifulsoup4 requests --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import torch
import requests
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset


In [3]:
# Load all CSVs files
true1 = pd.read_csv('sample_data/True.csv', encoding='utf-8', engine='python', on_bad_lines='skip')
true2 = pd.read_csv('sample_data/True_1.csv', encoding='utf-8', engine='python', on_bad_lines='skip')
fake1 = pd.read_csv('sample_data/Fake.csv', encoding='utf-8', engine='python', on_bad_lines='skip')
fake2 = pd.read_csv('sample_data/Fake_2.csv', encoding='utf-8', engine='python', on_bad_lines='skip')

# Assign labels
true1['label'] = 0
true2['label'] = 0
fake1['label'] = 1
fake2['label'] = 1

# Combine all into one DataFrame
df = pd.concat([true1, true2, fake1, fake2])[['text', 'label']].dropna()
df = df.sample(3000, random_state=42)  # optional downsampling for Colab


In [4]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)

train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})
val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [5]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }


In [6]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)



model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-6-a27ec958d0c6>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [7]:
trainer.train()
trainer.evaluate()


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.000300,0.031373,0.991667,0.991804,0.991667,0.991666
2,0.000100,0.016542,0.996667,0.996689,0.996667,0.996667


{'eval_loss': 0.01654158905148506,
 'eval_accuracy': 0.9966666666666667,
 'eval_precision': 0.9966888150609081,
 'eval_recall': 0.9966666666666667,
 'eval_f1': 0.9966666666666667,
 'eval_runtime': 258.5777,
 'eval_samples_per_second': 2.32,
 'eval_steps_per_second': 0.58,
 'epoch': 2.0}

In [8]:
def predict_with_label(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=256)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    label = torch.argmax(probs).item()

    result_msg = "REAL" if label == 0 else "FAKE"
    print(f"\n Prediction Result: {result_msg}")
    print(f"Confidence -> Real: {probs[0][0]:.4f}, Fake: {probs[0][1]:.4f}")

    return {
        "real_prob": float(probs[0][0]),
        "fake_prob": float(probs[0][1]),
        "predicted_label": label
    }


In [9]:
def extract_text_bs4(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        article = ' '.join([p.get_text() for p in paragraphs])
        return article.strip()
    except Exception as e:
        print("Error fetching article:", e)
        return ""


In [10]:
from urllib.parse import urlparse

def is_trusted_source(url):
    trusted_domains = ['bbc.com', 'reuters.com', 'apnews.com', 'npr.org']
    domain = urlparse(url).netloc
    return any(t in domain for t in trusted_domains)


In [11]:
def predict_from_url(url):
    print(f"\nFetching article from: {url}")
    text = extract_text_bs4(url)

    if not text or len(text.strip()) < 100:
        print("Unable to extract meaningful content from this URL.")
        return

    print("\nExtracted preview:\n", text[:400], "...\n")


    result = predict_with_label(text)


    if is_trusted_source(url) and result["predicted_label"] == 1:
        print("Trusted domain detected. This may be a false positive.")
        result["override"] = "real (trusted domain)"

    return result


In [32]:
predict_from_url("https://www.bbc.com/news/articles/c5y0y1py7ppo")


Fetching article from: https://www.bbc.com/news/articles/c5y0y1py7ppo

Extracted preview:
 More than 1,600 people have died following the devastating earthquake in Myanmar, with locals in some areas telling the BBC they had been left to dig through rubble for survivors with their bare hands. The quake has flattened much of Mandalay - the ancient capital and second-largest city which is home to about 1.5 million people. An acute lack of equipment, patchy communication networks and wrecke ...


 Prediction Result: FAKE
Confidence -> Real: 0.0010, Fake: 0.9990
Trusted domain detected. This may be a false positive.


{'real_prob': 0.0010057450272142887,
 'fake_prob': 0.9989942908287048,
 'predicted_label': 1,
 'override': 'real (trusted domain)'}